## SOUMAHORO Youssouf
## Enseignant: Olaf KOUAMO

### TP: Manipulation de DataFrame avec Spark

In [1]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f

In [1]:
from pyspark.sql import Row
import datetime

list_people = [(datetime.date(2018,1,3), 'Ankit',25),
     (datetime.date(2018,2,3), 'Jalfaizy',22),
     (datetime.date(2018,1,5), 'saurabh',20),
     (datetime.date(2018,1,12), 'Bala',26),
     (datetime.date(2018,7,9), 'Jules',19) ,
     (datetime.date(2018,3,18), 'Arild',43),
     (datetime.date(2018,1,5), 'sarah',20),
     (datetime.date(2018,8,12), 'Boly',33),
     (datetime.date(2018,4,6), 'Anita',35),
     (datetime.date(2018,12,6), 'Jules',22),
     (datetime.date(2018,7,24), 'Soul',20),
     (datetime.date(2018,6,17), 'Gral',54),
     (datetime.date(2018,9,7), 'Apoh',18),
     (datetime.date(2018,10,4), 'Dony',32),
     (datetime.date(2018,2,5), 'Tanoh',31),
     (datetime.date(2018,11,12), 'Issouf',27),
     (datetime.date(2018,10,3), 'Bilé',29),
     (datetime.date(2018,5,3), 'Gagnon',20),
     (datetime.date(2018,3,5), 'Papiss',28),
     (datetime.date(2018,2,12), 'Kravitz',34),
     (datetime.date(2018,5,9), 'Mouli',35),
     (datetime.date(2018,8,3), 'Jacques',27),
     (datetime.date(2018,12,5), 'soum',22),
     (datetime.date(2018,4,12), 'MBra',36)]

rdd = spark_session.sparkContext.parallelize(list_people)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2])))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

NameError: name 'spark_session' is not defined

In [4]:
schemaPeople.count()

24

In [24]:
schemaPeople.groupby('sexe').count().show()

+-----+-----+
| sexe|count|
+-----+-----+
|fille|   14|
|homme|   10|
+-----+-----+



In [46]:
df = schemaPeople.groupBy('sexe').agg(f.count("sexe").alias("nb_Pers"),
                                 f.mean("age").alias("moyenne"),
                                f.min("age").alias("minAge"),
                                f.max("age").alias("maxAge")).toPandas

In [58]:
# Nombre de fille
med_G = schemaPeople.filter(schemaPeople.sexe=="fille").approxQuantile("age", [0.5],0.000001)
med_F = schemaPeople.filter(schemaPeople.sexe=="homme").approxQuantile("age", [0.5],0.000001)
median = med_G+med_F

In [72]:
rdd = spark_session.sparkContext.parallelize(median)
p = rdd.map(lambda x: Row(median=x))
med = spark_session.createDataFrame(p)
med.show()

+------+
|median|
+------+
|  25.0|
|  28.0|
+------+



In [7]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
df = dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit'))

In [82]:
df.filter(df['days_since_last_visit'] < 400).count()

5

In [9]:
from pyspark.sql.functions import col,sum
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()

+---+----+----+---------------------+
|age|name|sexe|days_since_last_visit|
+---+----+----+---------------------+
|  0|   0|   0|                    0|
+---+----+----+---------------------+



In [4]:
from pyspark.sql.functions import udf
F1 = udf(lambda x: x.month)
schemaPeople.withColumn("mois",F1(schemaPeople["date"])).show(5)

+---+----------+--------+-----+----+
|age|      date|    name| sexe|mois|
+---+----------+--------+-----+----+
| 25|2018-01-03|   Ankit|fille|   1|
| 22|2018-02-03|Jalfaizy|fille|   2|
| 20|2018-01-05| saurabh|fille|   1|
| 26|2018-01-12|    Bala|fille|   1|
| 19|2018-07-09|   sarah|fille|   7|
+---+----------+--------+-----+----+
only showing top 5 rows



In [ ]:
# Autre methode
schemaPeople.withColumn("mois",f.month(schemaPeople["date"])).show(5)

In [105]:
schemaPeople.withColumn("adulte",f.when(schemaPeople["age"] < 25,0).when(schemaPeople["age"] >= 25,1)).show(5)

+---+----------+--------+-----+------+
|age|      date|    name| sexe|adulte|
+---+----------+--------+-----+------+
| 25|2018-01-03|   Ankit|fille|     1|
| 22|2018-02-03|Jalfaizy|fille|     0|
| 20|2018-01-05| saurabh|fille|     0|
| 26|2018-01-12|    Bala|fille|     1|
| 19|2018-07-09|   sarah|fille|     0|
+---+----------+--------+-----+------+
only showing top 5 rows

